### HEROKUのポスグレの接続テスト

In [ ]:
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()

users = os.getenv('HEROKU_PSGR_USER')    # DBにアクセスするユーザー名(適宜変更)
dbnames = os.getenv('HEROKU_PSGR_DATABASE')   # 接続するデータベース名(適宜変更)
passwords = os.getenv('HEROKU_PSGR_PASSWORD')  # DBにアクセスするユーザーのパスワード(適宜変更)
host = os.getenv('HEROKU_PSGR_HOST')     # DBが稼働しているホスト名(適宜変更)
port = 5432        # DBが稼働しているポート番号(適宜変更)

# PostgreSQLへ接続
conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)

# PostgreSQLにデータ登録
cursor = conn.cursor()


In [ ]:
import pandas as pd

cursor.execute('''SELECT * FROM maptable LIMIT 10''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(maptable_df)

cursor.execute('''SELECT * 
               FROM schedule 
               ORDER BY 取得時間 DESC
               LIMIT 10''')
cols = [col.name for col in cursor.description]
schedule_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(schedule_df)

